<a href="https://colab.research.google.com/github/Tihara-Jay/FYP-model-implementation/blob/main/Fine_tuning_of_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spam - 1 , ham - 0

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Flatten, Activation, BatchNormalization, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

Data preprocessing to make the data suitable to be passed into the VGG16 model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_PATH = '/content/drive/MyDrive/FYP/Dataset/Phishing'
ham_dir = '/content/drive/MyDrive/FYP/Dataset/Phishing/Ham'
spam_dir = '/content/drive/MyDrive/FYP/Dataset/Phishing/Spam'

In [ ]:
spam_files = os.listdir(spam_dir)
num_spam_files = len(spam_files)

ham_files = os.listdir(ham_dir)
num_ham_files = len(ham_files)

print("Number of spam images in spam_dir:", num_spam_files)
print("Number of ham images in ham_dir:", num_ham_files)

Number of spam images in spam_dir: 740
Number of ham images in ham_dir: 740


In [ ]:
ham_dir = '/content/drive/MyDrive/FYP/Dataset/Phishing/Ham'
spam_dir ='/content/drive/MyDrive/FYP/Dataset/Phishing/Spam'
model_dir = '/content/drive/MyDrive/FYP/Dataset/Phishing/Model'

In [ ]:
train_dir = model_dir + '/train'
test_dir = model_dir + '/test'
validation_dir = model_dir + '/validation'

Data augmentation

In [ ]:
trainAug = ImageDataGenerator (
  rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

#Initializaing the validation/testing data augmentation object (to which the mean subtraction will be added to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the mean subtraction value for each of the data augmentation objects

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [ ]:
BATCH_SIZE = 32
trainGen = trainAug.flow_from_directory(
	train_dir,
	class_mode="binary",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size = BATCH_SIZE)

valGen = valAug.flow_from_directory(
	validation_dir,
	class_mode="binary",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size= BATCH_SIZE)

testGen = valAug.flow_from_directory(
	test_dir,
	class_mode="binary",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size= BATCH_SIZE)
num_images = trainGen.n
print("Number of images:", num_images)

Found 1184 images belonging to 2 classes.
Found 148 images belonging to 2 classes.
Found 148 images belonging to 2 classes.
Number of images: 1184


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from keras import regularizers

baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001))(headModel)
headModel = Dropout(0.3)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
# placing the head FC model on top of the base model
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
baseModel.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
from imutils import paths
totalTrain = len(list(paths.list_images(train_dir)))
totalVal = len(list(paths.list_images(validation_dir)))
totalTest = len(list(paths.list_images(test_dir)))
print(totalTrain)
print(totalVal)
print(totalTest)

1184
148
148


steps per epoch = total number of samples / batch size

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_schedule = ReduceLROnPlateau(factor=0.1, patience=4, verbose=1)

In [ ]:
checkpoint_path = "/content/drive/MyDrive/FYP/Implementation/Attempt1/Checkpoints"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              verbose=1)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate= 1e-4),metrics=["accuracy"])

print("Training head...")
H = model.fit(
	x=trainGen,
	steps_per_epoch= totalTrain // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=  totalVal // BATCH_SIZE,
	epochs=50,
	callbacks=[early_stopping, lr_schedule, cp_callback])


Training head...
Epoch 1/50
37/37 [==============================] - ETA: 0s - loss: 5.7393 - accuracy: 0.5861 
Epoch 1: val_loss improved from inf to 3.55400, saving model to /content/drive/MyDrive/FYP/Implementation/Attempt1/Checkpoints
37/37 [==============================] - 749s 20s/step - loss: 5.7393 - accuracy: 0.5861 - val_loss: 3.5540 - val_accuracy: 0.6406 - lr: 1.0000e-04
Epoch 2/50
37/37 [==============================] - ETA: 0s - loss: 2.9806 - accuracy: 0.6334 
Epoch 2: val_loss improved from 3.55400 to 2.51601, saving model to /content/drive/MyDrive/FYP/Implementation/Attempt1/Checkpoints
37/37 [==============================] - 753s 20s/step - loss: 2.9806 - accuracy: 0.6334 - val_loss: 2.5160 - val_accuracy: 0.5938 - lr: 1.0000e-04
Epoch 3/50
37/37 [==============================] - ETA: 0s - loss: 2.0849 - accuracy: 0.6495 
Epoch 3: val_loss improved from 2.51601 to 1.66076, saving model to /content/drive/MyDrive/FYP/Implementation/Attempt1/Checkpoints
37/37 [======

In [ ]:
model.save('/content/drive/MyDrive/FYP/Implementation/Attempt1/21_03_vgg16_head.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/FYP/Implementation/Attempt1/head_model.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(testGen, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 1.0341585874557495
Test Accuracy: 0.75


https://pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_training(H, N):
    # construct a plot that plots the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.show()

Unfreezing the final set of conv layers in the initial base model

In [ ]:
trainGen.reset()
valGen.reset()

for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-8:]:
	layer.trainable = True

for layer in model.layers:
	print("{}: {}".format(layer, layer.trainable))

<keras.src.engine.input_layer.InputLayer object at 0x7c2980171600>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297fcd9960>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297fcda0e0>: False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c297fcdb070>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297fcdb6d0>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297fcdbf10>: False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c297e3fcf10>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297fcdb520>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297e3fdc00>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297e3fe710>: False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c297e3ff7c0>: False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c297e3ffd90>: False
<keras.src.layers.convolutional.conv

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate= 1e-4), metrics=["accuracy"])

H = model.fit(
	x=trainGen,
	steps_per_epoch = totalTrain // BATCH_SIZE,
	validation_data = valGen,
	validation_steps =  totalVal // BATCH_SIZE,
	epochs=50,
	callbacks=[early_stopping, lr_schedule,cp_callback ])

Epoch 1/50
37/37 [==============================] - ETA: 0s - loss: 0.7448 - accuracy: 0.6562
Epoch 1: val_loss did not improve from 0.59088
37/37 [==============================] - 42s 1s/step - loss: 0.7448 - accuracy: 0.6562 - val_loss: 0.6158 - val_accuracy: 0.7188 - lr: 1.0000e-04
Epoch 2/50
37/37 [==============================] - ETA: 0s - loss: 0.6386 - accuracy: 0.6486
Epoch 2: val_loss did not improve from 0.59088
37/37 [==============================] - 40s 1s/step - loss: 0.6386 - accuracy: 0.6486 - val_loss: 0.7606 - val_accuracy: 0.6406 - lr: 1.0000e-04
Epoch 3/50
37/37 [==============================] - ETA: 0s - loss: 0.6342 - accuracy: 0.6664
Epoch 3: val_loss did not improve from 0.59088
37/37 [==============================] - 38s 1s/step - loss: 0.6342 - accuracy: 0.6664 - val_loss: 0.6168 - val_accuracy: 0.7109 - lr: 1.0000e-04
Epoch 4/50
37/37 [==============================] - ETA: 0s - loss: 0.6263 - accuracy: 0.6596
Epoch 4: val_loss did not improve from 0.5908

In [ ]:
model.save('/content/drive/MyDrive/FYP/Implementation/Attempt1/Accuracy/bat_VGG16_finetune.h5')

In [ ]:
testGen.reset()
test_loss, test_accuracy = model.evaluate(testGen, verbose=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

5/5 [==============================] - 2s 400ms/step - loss: 0.4894 - accuracy: 0.7635
Test Loss: 0.4893653988838196
Test Accuracy: 0.7635135054588318


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
image_path = '/content/drive/MyDrive/FYP/PSPD_Submission_20200601_w1833519/Demo_Images/Non_spam.jpeg'

In [ ]:
from tensorflow.keras.preprocessing import image

img = image.load_img(image_path, target_size=(224, 224))  # Resize the image to match model input size
img_array = image.img_to_array(img)  # Convert image to numpy array
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
predictions = model.predict(img_array)
print(predictions)

1/1 [==============================] - 1s 1s/step
[[0.00035218]]


In [ ]:
threshold = 0.5
if predictions > threshold:
    print("SPAM")
else:
    print("HAM")

HAM


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

testGen.reset()
predictions = model.predict(testGen)

predicted_labels = (predictions > 0.5).astype(int)

true_labels = testGen.classes

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

5/5 [==============================] - 4s 583ms/step
Accuracy: 0.75
Precision: 0.7466666666666667
Recall: 0.7567567567567568
F1 Score: 0.7516778523489932


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/FYP/Implementation/Attempt1/Accuracy/checkpts_bat_VGG16_finetune.h5')